In [74]:
# import sys
# sys.path.append(sys.path[0] + "/..")

import json
import openai
import os
import pandas as pd

In [2]:
# !pip install evaluate

In [81]:
_KEY = open('/Users/user/Desktop/openai_key.txt', 'r').read()
_CLIENT = openai.OpenAI(
    api_key=_KEY,
    timeout=10,
)
_SEED = 10
_MODEL = "gpt-4"
_MAX_TOKENS = 1500

_PROMPT = """
You will get a claim and evidence which SUPPORTS the claim. Your task is to adjust the evidence such that the evidence refutes what is stated in the claim. 
---
Examples: 
claim: The Ten Commandments is an epic film.
evidence: The Ten Commandments is a 1956 American biblical epic film produced , directed , and narrated by Cecil B. DeMille , shot in VistaVision -LRB- color by Technicolor -RRB- , and released by Paramount Pictures .
adjusted evidence: The ten commandments were given to Moses by God, there exists no movie of such name.

claim: Brad Pitt and his wife Angelina Jolie were a Hollywood couple.
evidence: Angelina is divorced from actors Jonny Lee Miller, Billy Bob Thornton and Brad Pitt. She has six children with Pitt, three of whom were adopted internationally.
adjusted evidence: Brad and Angelina are a happily married Hollywood couple that has six children, they never got divorced.
---
Please complete: 
claim: {}
evidence: {}
adjusted evidence:"""

_PROMPT_REFORMULATION = """
You will get a claim and evidence which supports or refutes the claim. Your task is to adjust the evidence without changing its stance towards the claim. 
----
Examples: 
claim: The Ten Commandments is an epic film.
evidence: The Ten Commandments is a 1956 American biblical epic film produced , directed , and narrated by Cecil B. DeMille , shot in VistaVision -LRB- color by Technicolor -RRB- , and released by Paramount Pictures .
adjusted evidence: The American movie Ten Commandments is an epic film.

claim: Brad Pitt and his wife Angelina Jolie were a Hollywood couple.
evidence: Angelina is divorced from actors Jonny Lee Miller, Billy Bob Thornton and Brad Pitt. She has six children with Pitt, three of whom were adopted internationally.
adjusted evidence: Angelina is divorced from Pitt with whom she has six children, out of them three have been adopted
----
Please complete: 
claim: Selena recorded music.
evidence: Selena signed with EMI Latin in 1989 and released her self-titled debut album the same year , while her brother became her principal music producer and songwriter .
adjusted evidence:"""

POS_SAMPLE = {
    "claim": "",
    "reference": "",
    "target": "",
    "score": 1,
    "label": None
}
_EVIDENCE_FORMAT = "Title: {}; {}"


In [82]:
def _query_openai(prompt: str, client, keys=None, seed=_SEED, model=_MODEL, max_tokens=_MAX_TOKENS,
                  response_format="text"):
    return client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        model=model,
        max_tokens=max_tokens,
        response_format={"type": response_format},
        seed=seed,
    )

def _get_response_text(response: openai.types.chat.chat_completion.ChatCompletion):
    return response.choices[0].message.content


def _get_wiki_evidence(wiki_id: str, sentence_idx: int): 
    entry = wiki_db[wiki_db.id==wiki_id]
    sentences = list(entry.to_dict()["lines"].values())[0]
    sentence = sentences.split("\n")[sentence_idx]
    return sentence.split("\t")[1]
    

def _get_evidence_text(evidence_set: list):
    evidence_text = []
    for evidence in evidence_set:
        sentence = _get_wiki_evidence(evidence[2], evidence[3])
        evidence_text.append(_EVIDENCE_FORMAT.format(evidence[2], sentence))
        
    return " ".join(evidence_text)

def _format_claim_evidence(claim, evidence_set):
    return claim + " " + _get_evidence_text(evidence_set)

def load_jsonl_file(file_path, dataclass=None):
    content = []
    with open(file_path, "r", encoding="utf-8") as f:
        for entry in f.readlines():
            if dataclass:
                content.append(dacite.from_dict(data_class=dataclass, data=json.loads(entry)))
            else:
                content.append(json.loads(entry))
    return content

def _load_wiki_db(wiki_db_path = "/Users/user/Downloads/wiki-pages/wiki-pages-jsons"):
    wiki_entries = {}
    for file in os.listdir(wiki_db_path):
        with open(os.path.join(wiki_db_path, file)) as f:
            entry = json.load(f)
            wiki_entries[entry['id']] = entry['lines']
            
    return wiki_entries


In [113]:
# load fever training data
fever_train_path = "/Users/user/Library/CloudStorage/OneDrive-King'sCollegeLondon/PycharmProjects/fc-evidence-evaluation/data/fever/paper_dev.jsonl"
train = load_jsonl_file(fever_train_path)

# filter entries with more than one evidence set
train_multi_evidence = [entry for entry in train if len(entry['evidence'])>1]
train_multi_evidence_supports = [entry for entry in train_multi_evidence if entry['label']=='SUPPORTS']
train_multi_evidence_refutes = [entry for entry in train_multi_evidence if entry['label']=='REFUTES']

# Wikipedia db
wiki_db = pd.read_csv("/Users/user/Downloads/wiki-pages/wiki-pages.csv")

In [114]:
def _get_title_formatted(evidence_set):
    return "Title {}; ".format(evidence_set[0][2])

In [115]:
len(train_multi_evidence_supports)

917

In [117]:
# iterate over all fever samples
bleurt_training_data = []

for i, entry in enumerate(train_multi_evidence_supports):
    try:
        claim = entry['claim']
        reference = _get_evidence_text(entry["evidence"][0])
        target = _get_evidence_text(entry["evidence"][1])
        if target != reference:
            prompt = _PROMPT.format(claim, target)
            adj_evidence = _get_response_text(_query_openai(prompt, _CLIENT))

            new_entry = POS_SAMPLE.copy()
            new_entry['claim'] = claim
            new_entry['reference'] = claim + " " + reference
            new_entry['target'] = claim + " " + _get_title_formatted(entry["evidence"][1]) + adj_evidence
            new_entry['score'] = 0

            bleurt_training_data.append(new_entry)
    except Exception as e:
#         print(f"i: {i}; e: {e}")
        continue


In [120]:
len(bleurt_training_data)

363

In [119]:
# save bleurt training data as jsonl file
def to_dict(obj):
    return json.loads(json.dumps(obj, default=lambda o: o.__dict__))

def save_jsonl_file(data, file_path):
    with open(file_path, "w", encoding="utf-8") as f:
        for entry in data:
            json.dump(to_dict(entry), f)
            f.write("\n")

training_data_path = "/Users/user/Library/CloudStorage/OneDrive-King'sCollegeLondon/PycharmProjects/fc-evidence-evaluation/data/reference_scorer_training_data"

save_jsonl_file(bleurt_training_data, os.path.join(training_data_path, "fever_dev_gpt_score_zero.jsonl"))
